<a href="https://colab.research.google.com/github/sli782/pho/blob/master/MIE1624_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# Imports
from bs4 import BeautifulSoup
import requests, json
import pandas as pd

In [3]:
# Scraping parameters
fetching_list = {
    'job_title':'Data Scientist',
    'location':'Canada',
    'pages':5 
}

In [4]:
def create_url(parameters):
    # create base url for all further searches
    title = parameters['job_title'].replace(" ","+")
    place = parameters['location'].replace(" ","+")
    base_url = f"https://www.indeed.ca/jobs?q={title}&l={place}"
    return base_url

In [5]:
def get_desciption(j_title, j_soup, parameters):
    # # rate job by keywords
    
    try:
      description = j_soup.find(id="jobDescriptionText").get_text()
    except AttributeError:
      description = "Not Found" 
    # keywords = parameters['ordered_keywords']
    # title_keywords = parameters['title_keywords']
    # exclude_keywords = parameters['exclude_keywords']
    # total_keywords = len(keywords) + len(title_keywords)
    # keywords_present = []
    # title_keywords_present = []
    # rating = 0
    
    # # Check for keyword, add value to rating depending on ranking
    # for index,keyword in enumerate(keywords):
    #     if keyword in description:
    #         rating += len(keywords) - index
    #         keywords_present.append(keyword)
    
    # # Check for title keywords
    # for index,keyword in enumerate(title_keywords):
    #     if keyword in j_title:
    #         rating += total_keywords - index
    #         title_keywords_present.append(keyword)
    
    # # Normalise rating
    # rating = rating/sum(range(1,total_keywords+1))
    
    # # Check for excluded keywords
    # for keyword in exclude_keywords:
    #     if keyword in j_title:
    #         rating = 0
    #         break
    
    return description

In [6]:
def get_job_details(job,parameters):
    
    # Get link and title
    job_url = job.find(class_='title').a['href']
    
    # Correct for truncated URLs
    job_url = "https://www.indeed.ca" + job_url if (job_url.startswith("/")) else job_url
    job_page = requests.get(job_url)
    job_soup = BeautifulSoup(job_page.content,'html.parser')
    
    # Give URL after redirect (ads/analytics etc.)
    job_url = job_page.url 
    print(job_url)
    # Get job title and company name
    try:
      title = job.find(class_='title').a['title']
    except AttributeError:
      title = "Not Found "
    
    try:
      company = job_soup.find(class_="icl-u-lg-mr--sm").get_text()
    except AttributeError:
      company = "Not Found "
    
    
    # Get description, rating and present keywords
    description = get_desciption(title,job_soup,parameters)
    print(title, " ", company)
    return title, company, job_url, description

In [7]:
def scrape(parameters):
    
    # Create base url for all further searches
    base_url = create_url(parameters)
    
    # Output list and frame
    output = []
    
    for x in range(0,parameters['pages']):
        if(x==0):
            page_append = ""
        else: 
            page_append = "&start=" + str(x*10)
            
        # get page
        current_page = requests.get(base_url+page_append,timeout=5)
        page_soup = BeautifulSoup(current_page.content,"html.parser")
        
        for job in page_soup.select(".jobsearch-SerpJobCard"):
            title, company, url, description= get_job_details(job,parameters)
            output.append([title,company,description,url,x+1])
            
        print(f"Page {x+1} completed",end="\r")
        
    df_output_frame = pd.DataFrame(
        output,columns=['Job Title','Company','Description','Job URL''Page Found']).sort_values(
        by='Job Title',ascending=False).reset_index(drop=True)

    return df_output_frame
        

In [ ]:
jobs = scrape(fetching_list)

https://ca.indeed.com/viewjob?jk=5321f7ac707aaede&from=serp&vjs=3
Data Scientist   Technical Safety BC
https://ca.indeed.com/viewjob?jk=2367108563271394&from=serp&vjs=3
20-78 Data Scientist (Closing Date: November 22, 2020)   Technical Safety BC


In [ ]:
display(jobs.head())

# Output to Excel

In [ ]:
with pd.ExcelWriter('Excel Output.xlsx',options={'strings_to_urls': False}) as writer:
    jobs.to_excel(writer,index=False)

# Example output

<img src="img/Example_Output.PNG">